Import data from wiki

In [5]:
#!conda install -c conda-forge requests --yes
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [7]:
My_table = soup.find('table',{'class':'wikitable sortable'})
#My_table

Get dataframe with wiki data

In [8]:
import pandas as pd
my_list=[]
str_postcode=[]
for row in My_table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 3:
        postcode = cells[0].find(text=True)
        borough = cells[1].find(text=True)
        neighbourhood = cells[2].find(text=True)
        my_list.append([postcode,borough,neighbourhood])
df=pd.DataFrame(my_list,columns=['PostCode','Borough','Neighbourhood'])
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean data

In [9]:
#delete row where Borough is 'Not assigned'
df_cleaned=df[df['Borough']!='Not assigned']
#print (df_cleaned.head())
#print (df_cleaned.shape)
#df_cleaned

#replace 'Not assigned' of neighbourhood with borough value
#print (df_cleaned[df_cleaned['Neighbourhood']=='Not assigned'])
#print (df_cleaned[df_cleaned['PostCode']=='M7A'])
df_cleaned.at[8,'Neighbourhood'] = "Queen's Park"
#df_cleaned[df_cleaned['PostCode']=='M7A']

In [22]:
df_final=df_cleaned.groupby('PostCode')['Neighbourhood'].apply(', '.join).reset_index()
df_final = df_final.replace('\n','', regex=True)
df_final=df_final.merge(df_cleaned[['PostCode','Borough']],how='left',on='PostCode')
df_final = df_final.drop_duplicates(subset='PostCode').reset_index(drop=True)
#change sequence of columns
columnsTitles = ['PostCode', 'Borough', 'Neighbourhood']
df_final = df_final.reindex(columns=columnsTitles)
#df_final

Get size of dataframe

In [221]:
df_final.shape

(103, 3)

Read geoinfo from csv file and combine it with the above dataframe

In [11]:
geo_info=pd.read_csv('Geospatial_Coordinates.csv')
geo_info.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Show the final dataframe

In [23]:
my_df = df_final.merge(geo_info,how='left',left_on='PostCode',right_on='Postal Code')
my_df=my_df.drop(columns='Postal Code')
#print (my_df[my_df['PostCode']=='M9V'])
my_df.head(10)

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [118]:
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#get sub dataframe with Toronro in borough
ind = my_df['Borough'].str.contains("Toronto")
df_toronto = my_df[ind].reset_index()
latitude = df_toronto.loc[0,'Latitude']
longitude = df_toronto.loc[0,'Longitude']
#print ([latitude,longitude])
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
df_toronto=df_toronto.drop('index',1)
#print (df_toronto.head())
map_Toronto

![title](distribution.png)

Utilizing the Foursquare API to explore the neighborhoods and segment them

In [21]:
CLIENT_ID = '15VFWJCTZE4GGMH354ZIZBZJQ5RW1LMSCYFWQY4UVWQERT54' # your Foursquare ID
CLIENT_SECRET = 'WMPYUZYKZ1XRDQEKBNC5U302NMJXKRE4QACXFOFU5JLWBXDR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

Explore all the venues in the neighbourhood of Toronto area

In [27]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [37]:
print(toronto_venues.shape)
toronto_venues.head()

(1700, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [47]:
#toronto_venues.groupby('Neighborhood').count()
print (len(toronto_venues['Venue Category'].unique()))  #how many different kinds of Venue Categories
toronto_venues.head()

238


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Analyze nerghbourhood

In [66]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
columnsTitles = toronto_onehot.columns.tolist()
ind=columnsTitles.index('Neighborhood')
new_columns = ['Neighborhood']+columnsTitles[:ind]+columnsTitles[ind+1:]
toronto_onehot = toronto_onehot.reindex(columns=new_columns)
print (toronto_onehot.shape)
toronto_onehot.head()


(1700, 238)


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [69]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print (toronto_grouped.shape)
toronto_grouped.head()

(38, 238)


,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0,0.076923,0.076923,0.076923,0.153846,0.153846,0.153846,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


Cluster the neighborhoods in Toronto area 

In [152]:
#decide clustering groups
tmp=len(df_toronto['Borough'].unique())
print ('There are',tmp,'boroughs included in our dataset')
kclusters =4
print ('This area will be clustered into',kclusters,'parts')

There are 4 boroughs included in our dataset
This area will be clustered into 4 parts


In [153]:
from sklearn.cluster import KMeans
#use kmean to do clustering
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head()
sum(toronto_grouped_clustering.dtypes == 'float64') #check if all are numerical data
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Visualize the final result

In [154]:
#toronto_grouped.insert(0,'Cluster Labels', kmeans.labels_)
toronto_merged = df_toronto
#toronto_merged
toronto_merged=toronto_merged.merge(toronto_grouped[['Cluster Labels','Neighborhood']],how='left',left_on='Neighbourhood',right_on='Neighborhood')
toronto_merged=toronto_merged.drop('Neighborhood',1)

In [156]:
import numpy as np
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

![title](clustering.png)